In [1]:
# preprocessing data to fit in DVI/TimeVis's format
# import modules
import sys
import os
import numpy as np
import torch
import json
import collections

/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONTENT_PATH = "/home/yifan/0ExpCifar10/googlenet/vis"

In [4]:
sys.path.append(CONTENT_PATH)
sys.path.append(CONTENT_PATH)
VIS_METHOD = "DVI"
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]
import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))
model_path = os.path.join(CONTENT_PATH, "Model")

In [5]:
index = list(range(LEN))
EPOCH_PERIOD = 1
for n_epoch in range(EPOCH_START,EPOCH_END + EPOCH_PERIOD, EPOCH_PERIOD):
    m_path = os.path.join(model_path, "epoch={:03d}.ckpt".format(n_epoch-1))
    save_param =  torch.load(m_path)
    state_dict = save_param["state_dict"]

    order_dict =  collections.OrderedDict()
    for key in state_dict.keys():
        new_key = key.replace("model.","")
        order_dict[new_key] = state_dict[key]

    save_dir = os.path.join(model_path, "Epoch_{}".format((n_epoch) // EPOCH_PERIOD)) 
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    torch.save(order_dict, os.path.join(save_dir, "subject_model.pth"))
    with open(os.path.join(save_dir, "index.json"),"w") as f:
        json.dump(index, f)
    os.remove(m_path)

In [ ]:
# index = np.random.choice(list(range(TRAIN_NUM)), size=select_num, replace=False).tolist()
# test_idxs = np.random.choice(list(range(40000)), size=10000, replace=False).tolist()
# for n_epoch in range(EPOCH_START,EPOCH_END + EPOCH_PERIOD, EPOCH_PERIOD):

#     # state_dict =  torch.load(os.path.join(model_path, "epoch={:03d}.ckpt".format(n_epoch)))["state_dict"]
#     #
#     # order_dict =  collections.OrderedDict()
#     # for key in state_dict.keys():
#     #     new_key = key.replace("model.","")
#     #     order_dict[new_key] = state_dict[key]

#     save_dir = os.path.join(model_path, "Epoch_{}".format((n_epoch) // EPOCH_PERIOD)) # TODO
#     makedir_exist_ok(save_dir)
#     # torch.save(order_dict, os.path.join(save_dir, "subject_model.pth"))
#     with open(os.path.join(save_dir, "index.json"),"w") as f:
#         json.dump(index, f)
#     with open(os.path.join(save_dir, "test_index.json"),"w") as f:
#         json.dump(test_idxs, f)
